# Compute Antarctic, depth integrated, cross slope heat transport, online terms

In [2]:
%matplotlib inline

import cosima_cookbook as cc
from cosima_cookbook import distributed as ccd
import matplotlib.pyplot as plt
import numpy as np
import netCDF4 as nc
import xarray as xr
import glob,os
import cmocean.cm as cmocean

import logging
logging.captureWarnings(True)
logging.getLogger('py.warnings').setLevel(logging.ERROR)

from dask.distributed import Client

In [3]:
#client = Client(n_workers=4)
# >> dask-scheduler
# >> dask-worker tcp://10.0.64.9:8786 --memory-limit 4e9 --nprocs 6 --nthreads 1 --local-directory /local/g40/amh157
#client = Client('tcp://10.0.64.9:8786', local_dir='/local/g40/amh157')
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 7
Total threads: 28,Total memory: 251.20 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:45969,Workers: 7
Dashboard: /proxy/8787/status,Total threads: 28
Started: Just now,Total memory: 251.20 GiB
Comm: tcp://127.0.0.1:45253,Total threads: 4
Dashboard: /proxy/40163/status,Memory: 35.89 GiB
Nanny: tcp://127.0.0.1:33201,


In [4]:
#tides_session = cc.database.create_session('/g/data/v45/akm157/jupyter_scripts/tides/ryf9091_tides.db')
session = cc.database.create_session()

In [23]:
year = '2095'
exp = '01deg_jra55v140_iaf_cycle3'
expt = '01deg_jra55v140_iaf_cycle3'
#exp = '01deg_jra55v13_ryf9091_tides_fixed'

start_time='1958-01-01'
end_time='1958-01-31'
time_period = str(int(start_time[:4]))+'-'+str(int(end_time[:4]))

# reference density value:
rho_0 = 1035.0
# specific heat capacity of sea water:
cp = 3992.1
lat_range = slice(-90,-59)

isobath_depth = 1000

# pick a freezing point temp:
temp_freezing = -3.82

## Open contour data, extract lat/lon on contour

In [10]:
cutout_latind=510

In [11]:
		########### Import Adeles 1km contour on X,Y grid                
		outfile = '/g/data/v45/akm157/model_data/access-om2/Antarctic_slope_contour_'+str(isobath_depth)+'m.npz'
		data = np.load(outfile)
		mask_y_transport = data['mask_y_transport']; 
		mask_y_transport=mask_y_transport[:cutout_latind,:]       ##############################
		mask_x_transport = data['mask_x_transport']; 
		mask_x_transport=mask_x_transport[:cutout_latind,:]		################################
		mask_y_transport_numbered = data['mask_y_transport_numbered']
		mask_y_transport_numbered=mask_y_transport_numbered[:cutout_latind,:]		################################
		mask_x_transport_numbered = data['mask_x_transport_numbered']
		mask_x_transport_numbered=mask_x_transport_numbered[:cutout_latind,:]		################################
		
		
		#cutting the matrices
		ylength= np.shape(mask_x_transport)[0]
		
		
		yt_ocean0 = cc.querying.getvar(expt,'yt_ocean',session,n=1)
		yt_ocean0 = yt_ocean0.isel(yt_ocean=slice(0,ylength))
		yu_ocean0 = cc.querying.getvar(expt,'yu_ocean',session,n=1)
		yu_ocean0 = yu_ocean0.isel(yu_ocean=slice(0,ylength))
		xt_ocean0 = cc.querying.getvar(expt,'xt_ocean',session,n=1)
		xu_ocean0 = cc.querying.getvar(expt,'xu_ocean',session,n=1)


		
		mask_x_transport =xr.DataArray(data['mask_x_transport']).assign_coords({"dim_0": np.array(yt_ocean0),"dim_1": np.array(xu_ocean0)}).rename(dim_0="y_ocean",dim_1="x_ocean")
		mask_x_transport=mask_x_transport[:cutout_latind,:]       ##############################
		mask_y_transport =xr.DataArray(data['mask_y_transport']).assign_coords({"dim_0": np.array(yt_ocean0),"dim_1": np.array(xu_ocean0)}).rename(dim_0="y_ocean",dim_1="x_ocean")
		mask_y_transport=mask_y_transport[:cutout_latind,:]		################################		
		mask_x_transport_numbered =xr.DataArray(data['mask_x_transport_numbered']).assign_coords({"dim_0": np.array(yt_ocean0),"dim_1": np.array(xu_ocean0)}).rename(dim_0="y_ocean",dim_1="x_ocean")          ##################
		mask_x_transport_numbered=mask_x_transport_numbered[:cutout_latind,:]		################################		
		mask_y_transport_numbered =xr.DataArray(data['mask_y_transport_numbered']).assign_coords({"dim_0": np.array(yu_ocean0),"dim_1": np.array(xt_ocean0)}).rename(dim_0="y_ocean",dim_1="x_ocean")          ##################
		mask_y_transport_numbered=mask_y_transport_numbered[:cutout_latind,:]		################################
		
		num_points = int(np.maximum(np.max(mask_y_transport_numbered),np.max(mask_x_transport_numbered)))                

In [14]:
		#cutting the matrices
		ylength= np.shape(mask_x_transport)[0]
		
		
		yt_ocean = cc.querying.getvar(expt,'yt_ocean',session,n=1)
		yt_ocean = yt_ocean.isel(yt_ocean=slice(0,ylength))
		yu_ocean = cc.querying.getvar(expt,'yu_ocean',session,n=1)
		yu_ocean = yu_ocean.isel(yu_ocean=slice(0,ylength))
		xt_ocean = cc.querying.getvar(expt,'xt_ocean',session,n=1)
		xu_ocean = cc.querying.getvar(expt,'xu_ocean',session,n=1)

In [17]:
lat_along_contour = np.zeros((num_points))
lon_along_contour = np.zeros((num_points))

# locations for zonal transport:
x_indices_masked = mask_x_transport_numbered.stack().values
x_indices = np.sort(x_indices_masked[x_indices_masked>0])
for count in x_indices:
    count = int(count)
    jj = int(np.where(mask_x_transport_numbered==count)[0])
    ii = int(np.where(mask_x_transport_numbered==count)[1])   
    lon_along_contour[count-1] = xu_ocean[ii].values
    lat_along_contour[count-1] = mask_x_transport_numbered.y_ocean[jj].values
    
# locations for meridional transport:
y_indices_masked = mask_y_transport_numbered.stack().values
y_indices = np.sort(y_indices_masked[y_indices_masked>0])
for count in y_indices:
    count = int(count)
    jj = np.where(mask_y_transport_numbered==count)[0]
    ii = np.where(mask_y_transport_numbered==count)[1]
    lon_along_contour[count-1] = mask_x_transport_numbered.x_ocean[ii].values
    lat_along_contour[count-1] = yu_ocean[jj].values

In [18]:
lat_along_contour

array([-66.00806249, -66.00806249, -66.00806249, ..., -66.05030184,
       -66.05030184, -66.02918216])

## Compute heat transports offline, and saves it

Importing temperatures

In [24]:

	temp = cc.querying.getvar(exp,'temp',session,ncfile='%daily%',start_time=start_time,
	                           end_time=end_time).sel(time=slice(start_time,end_time))
	temp = temp.sel(yt_ocean=lat_range)
	
	temp_s = temp.copy()
	temp_s=temp_s.rename(yt_ocean='y_ocean',xt_ocean='x_ocean') #added by wilton
	temp_s.coords['y_ocean'] = np.array(yu_ocean)
	
	temp_n = temp.roll(yt_ocean=-1)
	temp_n=temp_n.rename(yt_ocean='y_ocean',xt_ocean='x_ocean') #added by wilton
	temp_n.coords['y_ocean'] = np.array(yu_ocean)
	# # temp_ygrid will be on the vhrho grid:
	temp_ygrid = (temp_s + temp_n)/2
	


Importing transports

In [31]:
	
	vhrho = cc.querying.getvar(exp,'vhrho_nt',session,ncfile='%daily%',start_time=start_time,end_time=end_time).sel(time=slice(start_time,end_time))
	
	# select latitude range and this month:
	vhrho = vhrho.sel(yt_ocean=lat_range)


temp_yflux (withotua accounting for the freezing point) will be

In [32]:
	vhrho = vhrho.rename({'yt_ocean':'y_ocean', 'xt_ocean':'x_ocean'})
	
	# vhrho and uhrho are saved with the wrong coords, change them:
	vhrho.coords['y_ocean'] = np.array(yu_ocean)
	
	
	
	# compute heat transport and depth sum:
	temp_yflux = (temp_ygrid*vhrho)
	
	
	### Convert to transports 
	
	# First we also need to change coords on dxu, dyt, so we can multiply the transports:
	dxu = cc.querying.getvar('01deg_jra55v13_ryf9091', 'dxu',session, n=1, ncfile = 'ocean_grid.nc')# getting from RYF
	# select latitude range:
	dxu = dxu.sel(yu_ocean=lat_range)
	# remove geolon_t/geolat_t coordinates:
	dxu = dxu.reset_coords().dxu 
	dxu.coords['xu_ocean'] = xt_ocean.values
	dxu = dxu.rename({'yu_ocean':'y_ocean', 'xu_ocean':'x_ocean'}) 


    
	# convert to transport:
	temp_yflux = temp_yflux*dxu/rho_0


Now for temp_xflux

In [36]:
 
	# interpolate temperature to uhrho grids:
	
	temp = cc.querying.getvar(exp,'temp',session,ncfile='%daily%',start_time=start_time,end_time=end_time).sel(time=slice(start_time,end_time))
	temp = temp.sel(yt_ocean=lat_range)
	
	# interpolate to correct grid:
	temp = temp.rename({'yt_ocean':'y_ocean', 'xt_ocean':'x_ocean'}) 
	temp_w = temp.copy()
	temp_w.coords['x_ocean'] = xu_ocean.values
	temp_e = temp.roll(x_ocean=-1)
	temp_e.coords['x_ocean'] = xu_ocean.values
	# temp_xgrid will be on the uhrho grid:
	temp_xgrid = (temp_e + temp_w)/2
	
	
	uhrho = cc.querying.getvar(exp,'uhrho_et',session,ncfile='%daily%',start_time=start_time,end_time=end_time).sel(time=slice(start_time,end_time))
	
	# select latitude range and this month:
	uhrho = uhrho.sel(yt_ocean=lat_range)
	
	# Note that vhrho is defined as the transport across the northern edge of a tracer cell so its coordinates 
	#       should be (yu_ocean, xt_ocean).
	#  uhrho is defined as the transport across the eastern edge of a tracer cell so its coordinates should 
	#       be (yt_ocean, xu_ocean).
	#  However we will keep the actual name as simply y_ocean/x_ocean irrespective of the variable
	#       to make concatenation and sorting possible.
	uhrho = uhrho.rename({'yt_ocean':'y_ocean', 'xt_ocean':'x_ocean'})
	
	# vhrho and uhrho are saved with the wrong coords, change them:
	uhrho.coords['x_ocean'] = xu_ocean.values
	
	
	
	
	# compute heat transport and depth sum:
	temp_xflux = (temp_xgrid*uhrho)
	
	
	### Convert to transports 
	
	# First we also need to change coords on dxu, dyt, so we can multiply the transports:
	dyt = cc.querying.getvar('01deg_jra55v13_ryf9091', 'dyt',session, n=1, ncfile = 'ocean_grid.nc')
	# select latitude range:
	dyt = dyt.sel(yt_ocean=lat_range)
	# remove geolon_t/geolat_t coordinates:
	dyt = dyt.reset_coords().dyt
	dyt.coords['xt_ocean'] = xu_ocean.values
	dyt = dyt.rename({'yt_ocean':'y_ocean','xt_ocean':'x_ocean'})
	
	# convert to transports:
	temp_xflux = temp_xflux*dyt/rho_0

Saving these in case we want to see them in the future

In [42]:
temp_xflux = temp_xflux.rename(y_ocean='yt_ocean',x_ocean='xu_ocean')
temp_yflux = temp_yflux.rename(y_ocean='yu_ocean',x_ocean='xt_ocean')

In [45]:
temp_yflux.time


<xarray.DataArray 'time' (time: 31)>
array(['1958-01-01T12:00:00.000000000', '1958-01-02T12:00:00.000000000',
       '1958-01-03T12:00:00.000000000', '1958-01-04T12:00:00.000000000',
       '1958-01-05T12:00:00.000000000', '1958-01-06T12:00:00.000000000',
       '1958-01-07T12:00:00.000000000', '1958-01-08T12:00:00.000000000',
       '1958-01-09T12:00:00.000000000', '1958-01-10T12:00:00.000000000',
       '1958-01-11T12:00:00.000000000', '1958-01-12T12:00:00.000000000',
       '1958-01-13T12:00:00.000000000', '1958-01-14T12:00:00.000000000',
       '1958-01-15T12:00:00.000000000', '1958-01-16T12:00:00.000000000',
       '1958-01-17T12:00:00.000000000', '1958-01-18T12:00:00.000000000',
       '1958-01-19T12:00:00.000000000', '1958-01-20T12:00:00.000000000',
       '1958-01-21T12:00:00.000000000', '1958-01-22T12:00:00.000000000',
       '1958-01-23T12:00:00.000000000', '1958-01-24T12:00:00.000000000',
       '1958-01-25T12:00:00.000000000', '1958-01-26T12:00:00.000000000',
       '1958-01-27T12:00:00.000000000', '1958-01-28T12:00:00.000000000',
       '1958-01-29T12:00:00.000000000', '1958-01-30T12:00:00.000000000',
       '1958-01-31T12:00:00.000000000'], dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 1958-01-01T12:00:00 ... 1958-01-31T12:00:00
Attributes:
    long_name:       time
    cartesian_axis:  T
    calendar_type:   GREGORIAN
    bounds:          time_bounds

In [59]:
np.array(xu_ocean[1]),np.array(xt_ocean[1])

(array(-279.8), array(-279.85))

In [65]:

		# PREVIOUS SAVEDIR TO LOCATE THE FILES save_dir  = '/g/data/x77/wf4500/ASC_project/ASC_speed/monthly/OM2_RYF/'
		save_dir  = '/g/data/x77/wf4500/ASC_project/Offline_tempflux/IAF/'        
		file_name = 'temp_flux'
		
		data_x=xr.DataArray((temp_xflux),name="temp_xflux",dims=["time","st_ocean","yt_ocean","xu_ocean"])
		data_y=xr.DataArray((temp_yflux),name="temp_yflux",dims=["time","st_ocean","yu_ocean","xt_ocean"])
		data_z=xr.DataArray((temp_yflux.st_ocean),name="st_ocean",dims=["st_ocean"])
		data_xu=xr.DataArray((xu_ocean),name="xu_ocean",dims=["xu_ocean"])
		data_xt=xr.DataArray((xt_ocean),name="xt_ocean",dims=["xt_ocean"])
		data_yu=xr.DataArray((yu_ocean),name="yu_ocean",dims=["yu_ocean"])
		data_yt=xr.DataArray((yt_ocean),name="yt_ocean",dims=["yt_ocean"])
		data_time=xr.DataArray((temp_yflux.time),name="time",dims=["time"])
		#data_x,data_y,data_z,data_xu,data_xt,data_yu,data_yt,data_time
	
				
		data_to_saveuv = xr.merge([data_x,data_y])
		data_to_saveuv['st_ocean']=np.array(temp_yflux.st_ocean)
		data_to_saveuv['time']=temp_yflux.time
		data_to_saveuv
		#data_to_saveuv.to_netcdf(save_dir + file_name + start_time +"_xy.nc")




<xarray.Dataset>
Dimensions:     (xu_ocean: 3600, yt_ocean: 510, st_ocean: 75, time: 31,
                 xt_ocean: 3600, yu_ocean: 510)
Coordinates:
  * xu_ocean    (xu_ocean) float64 -279.9 -279.8 -279.7 ... 79.8 79.9 80.0
  * yt_ocean    (yt_ocean) float64 -81.11 -81.07 -81.02 ... -59.14 -59.08 -59.03
  * st_ocean    (st_ocean) float64 0.5413 1.681 2.94 ... 5.511e+03 5.709e+03
  * time        (time) datetime64[ns] 1958-01-01T12:00:00 ... 1958-01-31T12:0...
  * xt_ocean    (xt_ocean) float64 -279.9 -279.8 -279.7 ... 79.75 79.85 79.95
  * yu_ocean    (yu_ocean) float64 -81.09 -81.05 -81.0 ... -59.11 -59.06 -59.01
Data variables:
    temp_xflux  (time, st_ocean, yt_ocean, xu_ocean) float32 dask.array<chunksize=(1, 19, 135, 180), meta=np.ndarray>
    temp_yflux  (time, st_ocean, yu_ocean, xt_ocean) float32 dask.array<chunksize=(1, 19, 135, 180), meta=np.ndarray>
Attributes:
    INFO:     Does not account freezing temperature transport

2023-09-25 12:26:46,929 - distributed.nanny - WARNING - Restarting worker
2023-09-25 12:26:46,996 - distributed.nanny - WARNING - Restarting worker
2023-09-25 12:26:47,288 - distributed.nanny - WARNING - Restarting worker
2023-09-25 12:26:48,295 - distributed.nanny - WARNING - Restarting worker


## Extract heat transport values along isobath contour:

## Convert cross-slope heat transport from isobath coordinate to longitude coordinate

## Zonal heat convergence

## Save cross-slope and zonal convergence terms for this isobath